In [ ]:
# Prereq libraries
import numpy as np
import pandas as pb
import random
from nltk import word_tokenize

# Possibly useful TensorFlow and Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM
import tensorflow as tf

# Read in dataset from csv files
dataset = pb.read_csv('min75_max5k_as_ids.csv')

# I/O file parameters
TRUE_TAGS_OUTPUT_FILE = "true_tags.csv"
PREDICTED_TAGS_OUTPUT_FILE = "predicted_tags.csv"
RANDOM_GAME_OUTPUT_FILE = "random_game_prediction.csv"

# Data set split parameters
TRAINING_SPLIT = 0.2
ALPHA = 0.01

In [ ]:
from functs import npFloatArray, load_data

# load features and labels based on those present in the CSV files
features = np.array(dataset.copy())

# Load Data
X_train, X_test, y_train, y_test = load_data(dataset, 
                                             TRAINING_SPLIT)

# Double check X_train's shape
print("X_train's Shape: " + str(X_train.shape[0]))

# Set up dataset for model fitting
num_rows, num_cols = X_train.shape
X_train = np.reshape(X_train, (X_train.shape[0],
                               1,
                               X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],
                             1,
                             X_test.shape[1]))

# Test code used for printing results
# ========================================================
# all_tests= []
# all_results = []

# random_game_index = np.random.randint(0, len(all_X))
# random_game_description = all_X[random_game_index]
# random_game_results = []
# ========================================================

# Test code for shape sanity checks
# ========================================================
# print(num_cols)
# print(num_rows)
# print(y_train.shape[1:][0])

# print(random_game_index)
# print(random_game_description.shape)
# print(X_train.shape[1:])
# ========================================================

In [ ]:
from matplotlib import pyplot as plt
from functs import myPrint
from keras.callbacks import CSVLogger

# RNN model implementation using Keras Library
# Inspired by https://www.youtube.com/watch?v=BSpXCRTOLJA

# Create sequential model object
model = Sequential()

# First input layer using LSTM with window size 64, train_x shape, & RelU activation [131 nodes]
model.add(LSTM(num_cols, input_shape = (X_train.shape[1:]),
               activation = 'relu', 
               return_sequences = True))
model.add(Dropout(0.2))

# First hidden layer using LSTM with RelU activation [512 nodes]
model.add(LSTM(int(512), 
               activation = 'relu'))
model.add(Dropout(0.2))

# Final output layer using softmax activation [131 nodes]
model.add(Dense(int(y_train.shape[1:][0]), 
                activation = 'softmax'))

# optimizer used for compilation, 'Adam' refers to gradient descent method that "adapts" using first and second order moments
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3, 
                               decay = 1e-5)

# Final compilation
# Metrics used: accuracy, precision, RMSE
model.compile(loss = 'mean_squared_error', 
              optimizer = opt, 
              metrics = ['accuracy', 
                         tf.keras.metrics.Precision(), 
                         tf.keras.metrics.RootMeanSquaredError()])

# Model Fitting process, this is the last step
csv_logger = CSVLogger('log.txt', 
                       append = True, 
                       separator = ';')
history = model.fit(X_train, 
                    y_train, 
                    epochs = 70, 
                    validation_data = (X_test, 
                                       y_test), 
                    callbacks = [csv_logger])

# Plot Measurment Results
# From: https://stackoverflow.com/questions/41908379/keras-plot-training-validation-and-test-set-accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
fig1 = plt.gcf()
plt.show()
fig1.savefig('Accuracy_Over_Time.png')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
fig2 = plt.gcf()
plt.show()
fig2.savefig('Loss_Over_Time.png')

# viewable summary
summary = model.summary(print_fn = myPrint)
